In [71]:
import os
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from numpy import genfromtxt
import pickle

<h1>Load Datasets</h1>

In [72]:
# Hyperparameters
INIT_PATH = '../'
TRAIN_RATIO = 0.7
AMOUNT_OF_PARAMETERS = 7

dataset = []
"""
static: 1,0,0,0,0,0,0
move: 0,1,0,0,0,0,0
rot: 0,0,1,0,0,0,0
frot: 0,0,0,1,0,0,0
light: 0,0,0,0,1,0,0
dynamiclight: 0,0,0,0,0,1,0
unknown: 0,0,0,0,0,0,1

"""
definition = ['static', 'move', 'rot', 'frot', 'light', 'dynamiclight','unknown']

def normalize(dataset):
    datasetTmp = np.copy(dataset)
    maxx = np.max(dataset, axis=0)
    minn = np.min(dataset, axis=0)
    rangee = maxx - minn
    datasetTmp -= minn
    datasetTmp /= rangee
    return datasetTmp

def loadDataset(path):
    dataset = (genfromtxt(INIT_PATH + path, delimiter=','))[1:,:]
    dataset = normalize(dataset)
    trainingAmount = int(TRAIN_RATIO * dataset.shape[0])
    training_dataset = dataset[0:trainingAmount,:]
    test_dataset = dataset[trainingAmount:,:]
    print("data: ", dataset.shape)
    return training_dataset.astype(float), test_dataset.astype(float)

def loadLabel(path):
#     labels = pickle.load(INIT_PATH + path)
    with open(INIT_PATH + path, 'rb') as handle:
        labels = pickle.load(handle)
    ###### TEMPORALLY
    del labels[-1]
    ######
    trainingAmount = int(TRAIN_RATIO * len(labels))
    print("labels: ", len(labels))
    training_labels = np.zeros((trainingAmount, AMOUNT_OF_PARAMETERS))
    test_labels = np.zeros((len(labels) - trainingAmount, AMOUNT_OF_PARAMETERS))
    label_counter = 0
    per_label = [0] * AMOUNT_OF_PARAMETERS
#     print(labels)
    for label in labels:
        tmp = np.zeros((1,AMOUNT_OF_PARAMETERS))
        if (label == "static"):
            tmp[0,0] = 1
            per_label[0] += 1
        elif (label == "move"):
            tmp[0,1] = 1
            per_label[1] += 1
        elif (label == "rot"):
            tmp[0,2] = 1
            per_label[2] += 1            
        elif (label == "frot"):
            tmp[0,3] = 1
            per_label[3] += 1
        elif (label == "light"):
            tmp[0,4] = 1
            per_label[4] += 1
        elif (label == "dynamiclight"):
            tmp[0,5] = 1
            per_label[5] += 1
        else:
            tmp[0,6] = 1
            per_label[6] += 1
        if(label_counter < trainingAmount):
            training_labels[label_counter,:] = tmp
        else:
            test_labels[label_counter - trainingAmount,:] = tmp
        label_counter += 1
    print(", ".join([definition[i] + ": " + str(per_label[i]) for i in range (len(per_label))]))
    return training_labels, test_labels
training_dataset, test_dataset = loadDataset("FrankensteinFullForm.csv")
training_labels, test_labels = loadLabel("LabelFrankensteinFullForm.p")

data:  (138, 10)
labels:  138
static: 22, move: 24, rot: 21, frot: 21, light: 23, dynamiclight: 22, unknown: 5


In [3]:
print(training_dataset.shape)
print(test_dataset.shape)
print(training_labels.shape)
print(test_labels.shape)
# training_labels

(96, 10)
(42, 10)
(96, 7)
(42, 7)


In [73]:
idx = np.random.permutation(len(data))
x,y = data[idx], classes[idx]

TypeError: only integer scalar arrays can be converted to a scalar index

In [4]:
from torch.utils.data import Dataset, DataLoader

class Data(Dataset):
    def __init__( self, data, labels, transform=None, data_dir=None):
        self.data = data
        self.labels = labels
        self.transform = transform
    def __getitem__(self,index):
        singleData = self.data[index]
        singleLabel = self.labels[index]
        return (singleData, singleLabel)
    def __len__(self):
        return len(self.data)
training_set = Data(training_dataset, training_labels)
test_set = Data(test_dataset, test_labels)
print(training_set.__getitem__(10))
print(training_set.__len__())

(array([0.1092296 , 0.16715095, 0.52934208, 0.21504995, 0.23992044,
       0.32241417, 0.31384785, 0.05240026, 0.41536227, 0.12281167]), array([1., 0., 0., 0., 0., 0., 0.]))
96


In [5]:
trainloader = DataLoader(training_set, batch_size = 4, shuffle = True)
dataiter = iter(trainloader)
datum,label = dataiter.next()
print(datum.shape)

testloader = DataLoader(test_set, batch_size = 4, shuffle = True)

torch.Size([4, 10])


In [6]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, output_dim)
#         self.fc1 = nn.Linear(input_dim, 100)
#         self.fc2 = nn.Linear(100, output_dim)
    def forward(self, x):
        x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
        return x
LRNet = LinearRegressionModel(10,7)
# LRNet = LRNet.cuda()
print(LRNet)

LinearRegressionModel(
  (fc1): Linear(in_features=10, out_features=7, bias=True)
)


In [19]:
import torch.optim as optim

criterion =  nn.MSELoss()
# optimizer = optim.SGD(LRNet.parameters(), lr = 0.001, momentum = 0.9)
optimizer = optim.SGD(LRNet.parameters(), lr = 0.001)

In [22]:
for epoch in range (50):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputt, labell = data
        inputs, labels = Variable(inputt.float()), Variable(labell.float())
#         print(inputs)
#         print(inputs.double())
        optimizer.zero_grad()
        outputs = LRNet(inputs)
#         if i == 1:
#             print('inputs: ',inputs[0])
#             print('outputs:', outputs[0])
#             print("labels:", labels[0])
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.data[0]
#         print('loss meme:',loss)
        if(i % 10 == 0):
            print('[%d, %5d] loss: %.3f' %
                (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0
print('finished training')

/home/bon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[1,     1] loss: 0.009
[1,    11] loss: 0.086
[1,    21] loss: 0.098
[2,     1] loss: 0.006
[2,    11] loss: 0.081
[2,    21] loss: 0.101
[3,     1] loss: 0.012
[3,    11] loss: 0.078
[3,    21] loss: 0.101
[4,     1] loss: 0.013
[4,    11] loss: 0.092
[4,    21] loss: 0.099
[5,     1] loss: 0.009
[5,    11] loss: 0.106
[5,    21] loss: 0.078
[6,     1] loss: 0.008
[6,    11] loss: 0.079
[6,    21] loss: 0.094
[7,     1] loss: 0.003
[7,    11] loss: 0.097
[7,    21] loss: 0.090
[8,     1] loss: 0.008
[8,    11] loss: 0.088
[8,    21] loss: 0.088
[9,     1] loss: 0.009
[9,    11] loss: 0.092
[9,    21] loss: 0.080
[10,     1] loss: 0.009
[10,    11] loss: 0.076
[10,    21] loss: 0.111
[11,     1] loss: 0.007
[11,    11] loss: 0.103
[11,    21] loss: 0.080
[12,     1] loss: 0.005
[12,    11] loss: 0.085
[12,    21] loss: 0.100
[13,     1] loss: 0.012
[13,    11] loss: 0.083
[13,    21] loss: 0.090
[14,     1] loss: 0.009
[14,    11] loss: 0.074
[14,    21] loss: 0.097
[15,     1] loss: 0

In [23]:
def outputToTensor(labels):
    # print(labels)
    # print(labels.numpy().astype(int))
    tmp = labels.numpy().astype(int)
    # print([ np.where(r==1)[0][0] for r in tmp ])
    tmp = [ np.where(r==1)[0][0] for r in tmp ]
    tmp = np.array(tmp)
    tmp = torch.from_numpy(tmp)
    # print(tmp)
    return tmp

In [41]:
definition = ['static', 'move', 'rot', 'frot', 'light', 'dynamiclight','unknown']

"""
static: 1,0,0,0,0,0,0
move: 0,1,0,0,0,0,0
rot: 0,0,1,0,0,0,0
frot: 0,0,0,1,0,0,0
light: 0,0,0,0,1,0,0
dynamiclight: 0,0,0,0,0,1,0
unknown: 0,0,0,0,0,0,1
"""


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))


dataiter  = iter(testloader)
test_data,labels = dataiter.next()
# show images
# imshow(torchvision.utils.make_grid(test_data))
# print labels
print("Ground truth : ", ' '.join(np.array2string(x.numpy()) for x in labels))
# print('dimensions : ',' '.join(x.numpy().shape for x in images))
# imshow(torchvision.utils.make_grid(test_data))

outputs = LRNet(Variable(test_data.float()))
# print('outputs:',outputs)
_,predicted = torch.max(outputs.data, 1)
print('Predicted torch : ',predicted )
print('Predicted : ',' '.join(definition[predicted[i]] for i in range(4)))

# print('Predicted :',' '.join('%5s'% classes[int(predicted[j])] for j in range(4) ))

Ground truth :  [0. 0. 0. 0. 0. 1. 0.] [0. 0. 0. 1. 0. 0. 0.] [0. 0. 0. 1. 0. 0. 0.] [0. 0. 0. 0. 1. 0. 0.]
Predicted torch :  tensor([ 4,  1,  4,  4])
Predicted :  light move light light


In [70]:
correct = 0
total = 0
i = 0
for data in testloader:
    i += 1
    images,labels = data
#     print('raw label : ',labels)
    outputs = LRNet(Variable(images.float()))
    _,predicted = torch.max(outputs.data, 1)
    print(predicted, labels.max(1)[1].long(), labels.max(1)[1].long() == predicted.long())
#     labels = outputToTensor(labels)
    total += labels.size(0)
#     print('predicted : ',predicted)
    compared = labels.max(1)[1].long() == predicted.long()
#     print('compare result : ', compared)
    correct += (compared).sum()    
print('Accuracy :',(correct/total*100))

tensor([ 3,  4,  4,  1]) tensor([ 2,  5,  2,  3]) tensor([ 0,  0,  0,  0], dtype=torch.uint8)
tensor([ 5,  4,  4,  1]) tensor([ 3,  6,  5,  2]) tensor([ 0,  0,  0,  0], dtype=torch.uint8)
tensor([ 4,  4,  4,  4]) tensor([ 5,  5,  5,  5]) tensor([ 0,  0,  0,  0], dtype=torch.uint8)
tensor([ 3,  4,  4,  5]) tensor([ 2,  5,  6,  3]) tensor([ 0,  0,  0,  0], dtype=torch.uint8)
tensor([ 4,  4,  4,  1]) tensor([ 3,  5,  4,  3]) tensor([ 0,  0,  1,  0], dtype=torch.uint8)
tensor([ 1,  3,  4,  4]) tensor([ 2,  2,  2,  3]) tensor([ 0,  0,  0,  0], dtype=torch.uint8)
tensor([ 4,  5,  4,  5]) tensor([ 5,  3,  5,  3]) tensor([ 0,  0,  0,  0], dtype=torch.uint8)
tensor([ 4,  4,  5,  4]) tensor([ 2,  2,  3,  5]) tensor([ 0,  0,  0,  0], dtype=torch.uint8)
tensor([ 4,  4,  1,  4]) tensor([ 2,  5,  3,  5]) tensor([ 0,  0,  0,  0], dtype=torch.uint8)
tensor([ 4,  5,  4,  5]) tensor([ 2,  3,  5,  3]) tensor([ 0,  0,  0,  0], dtype=torch.uint8)
tensor([ 4,  4]) tensor([ 4,  5]) tensor([ 1,  0], dtype=tor

In [227]:
tmp
tmp.max(1)[1]

tensor([ 2,  3,  5,  5])